In [1]:
import numpy as np
import pickle

### Join multiple runs (different pars for same size) into single dictionary

In [2]:
def copy_partial_data(d, D):
    if type(d) == dict:
        for k in d.keys():
            if k not in D:
                D[k] = d[k]
            else:
                D[k] = copy_partial_data(d[k], D[k])
    return D

In [3]:
Ps = np.arange(2, 15)
Ns = 8*Ps
N_cities =  np.arange(4, 65, 2)


dataset = "TSP" # "NPP" or "TSP"

N_idx = 11
if dataset == "NPP":
    vseeds = np.arange(42, 46)
    filestart = f"../data/SA_NPP/results-N={Ns[N_idx]}_P={Ps[N_idx]}"
elif dataset == "TSP":
    vseeds = [42]
    filestart = f"../data/SA_TSP/results-Nc={N_cities[N_idx]}"
M_strategies = ["feasibility", "optimality"]
temp_scalers = [1, 10, 100]
n_eta_runs = [.25, .5, .75]

global_D = {}

for vseed in vseeds:
    for Mstrat in M_strategies:
        for temp_scale in temp_scalers:
            for eta_run in n_eta_runs:
                filename = filestart + f"_pars_{N_idx}_{vseed}_{Mstrat}_{temp_scale}_{eta_run}.txt"
                file = open(filename, "rb")
                D = pickle.load(file)
                file.close()
                global_D = copy_partial_data(D, global_D)


file = open(filestart + ".txt", "wb")
pickle.dump(global_D, file)
file.close()
print(f"File {filestart}.txt written")

File ../data/SA_TSP/results-Nc=26.txt written


### Fix mistake for $N_{idx} = 7$

In [50]:
### correct the overwriting eta_index_0 of all the dictionaries

def correct_D_etarun(D, pars):
    k1 = "vseed_"+str(pars[0])
    k2 = "Tscale_"+str(pars[1])
    k3 = pars[2]
    if pars[3] == .25:
        k4 = "eta_run_0"
    elif pars[3] == .5:
        k4 = "eta_run_1"
    elif pars[3] == .75:
        k4 = "eta_run_2"

    data = D[k1][k2][k3].pop("eta_run_0")
    D[k1][k2][k3][k4] = data
    return D

N_idx = 7
vseeds = np.arange(42, 46)
M_strategies = ["feasibility", "optimality"]
temp_scalers = [1, 10, 100]
n_eta_runs = [.25, .5, .75]

filestart = f"../data/SA_NPP/results-N={Ns[N_idx]}_P={Ps[N_idx]}"
for vseed in vseeds:
    for Mstrat in M_strategies:
        for temp_scale in temp_scalers:
            for eta_run in n_eta_runs:
                filename = filestart + f"_pars_{N_idx}_{vseed}_{Mstrat}_{temp_scale}_{eta_run}.txt"
                file = open(filename, "rb")
                D = pickle.load(file)
                file.close()
                # correct D
                pars = (vseed, temp_scale, Mstrat, eta_run)
                D = correct_D_etarun(D, pars)

                # dump D in same filename but corrected
                filename = filestart + f"_pars_{N_idx}_{vseed}_{Mstrat}_{temp_scale}_{eta_run}_corrected.txt"
                file = open(filename, "wb")
                pickle.dump(D, file)
                file.close()

### Rename Dicts $"eta\_run\_i" \to "eta\_req\_\eta"$

In [ ]:
def correct_dict(D):
    for vseed in vseeds:
        for Mstrat in M_strategies:
            for temp_scale in temp_scalers:
                for eta_run, eta_req in enumerate(eta_reqs):
                    element = D["vseed_"+str(vseed)]["Tscale_"+str(temp_scale)][Mstrat].pop(f"eta_run_{eta_run}")
                    D["vseed_"+str(vseed)]["Tscale_"+str(temp_scale)][Mstrat][f"eta_req_{eta_req}"] = element
    return D
    

vseeds = np.arange(42, 46)
M_strategies = ["feasibility", "optimality"]
temp_scalers = [1, 10, 100]
eta_reqs = [.25, .5, .75]

for N_idx in range(8):
    filestart = f"../data/SA_NPP/results-N={Ns[N_idx]}_P={Ps[N_idx]}"
    file = open(filestart + ".txt", "rb")
    D = pickle.load(file)
    file.close()
    D_corr = correct_dict(D)

    file = open(filestart + "_corrected.txt", "wb")
    pickle.dump(D_corr, file)
    file.close()